<a href="https://colab.research.google.com/github/rbdus0715/Machine-Learning/blob/main/study/cuda/01.intro-cuda/08.memory_transfer_between_host_device.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

### **Host <-> Device**

- 128개의 스레드
- 2개의 스레드 블럭, 하나당 64개

|C|CUDA|
|-|-|
|malloc|cudaMalloc|
|memset|cudaMemset|
|free|cudaFree|

In [3]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
#include <cstring>
#include <time.h>

__global__ void mem_trs_test(int * input) {
    // 그리드 블럭과 스레드 블럭이 1차원이므로 gid는 다음과 같다.
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    printf("tid : %d, gid : %d, value : %d \n", threadIdx.x, gid, input[gid]);
}

int main() {

    int size = 128;
    int byte_size = size * sizeof(int);

    int * h_input;
    // 말록 함수는 void 포인터를 반환하므로 원하는 타입의 포인터로 캐스팅해야한다.
    h_input = (int*)malloc(byte_size);

    time_t t;
    srand((unsigned)time(&t));
    for(int i=0; i<size; i++) {
        h_input[i] = rand() & 0xff;
    }

    int * d_input;
    // 첫번째 인자로 더블 포인터 혹은 포인터에 포인터가 들어간다.
    cudaMalloc((void**)&d_input, byte_size);

    cudaMemcpy(d_input, h_input, byte_size, cudaMemcpyHostToDevice);

    dim3 block(64);
    dim3 grid(2);

    mem_trs_test<<<grid, block>>>(d_input);
    cudaDeviceSynchronize();

    cudaFree(d_input);
    free(h_input);

    cudaDeviceReset();
    return 0;
}

tid : 0, gid : 64, value : 253 
tid : 1, gid : 65, value : 232 
tid : 2, gid : 66, value : 210 
tid : 3, gid : 67, value : 181 
tid : 4, gid : 68, value : 62 
tid : 5, gid : 69, value : 19 
tid : 6, gid : 70, value : 37 
tid : 7, gid : 71, value : 24 
tid : 8, gid : 72, value : 233 
tid : 9, gid : 73, value : 216 
tid : 10, gid : 74, value : 17 
tid : 11, gid : 75, value : 25 
tid : 12, gid : 76, value : 183 
tid : 13, gid : 77, value : 250 
tid : 14, gid : 78, value : 48 
tid : 15, gid : 79, value : 67 
tid : 16, gid : 80, value : 65 
tid : 17, gid : 81, value : 58 
tid : 18, gid : 82, value : 167 
tid : 19, gid : 83, value : 116 
tid : 20, gid : 84, value : 184 
tid : 21, gid : 85, value : 221 
tid : 22, gid : 86, value : 222 
tid : 23, gid : 87, value : 160 
tid : 24, gid : 88, value : 46 
tid : 25, gid : 89, value : 65 
tid : 26, gid : 90, value : 200 
tid : 27, gid : 91, value : 167 
tid : 28, gid : 92, value : 191 
tid : 29, gid : 93, value : 172 
tid : 30, gid : 94, value : 91 


### **입력 데이터가 커널이 정한 데이터보다 클 때 (넘칠 때)**
- 배열 크기를 배열과 같이 넘겨줌으로써 if(gid < size)를 통해 해결

In [4]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
#include <cstring>
#include <time.h>

__global__ void mem_trs_test2(int * input, int size) {
    int gid = blockIdx.x * blockDim.x + threadIdx.x;

    if(gid < size)
        printf("tid : %d, gid : %d, value : %d \n", threadIdx.x, gid, input[gid]);
}

int main() {

    int size = 150;
    int byte_size = size * sizeof(int);

    int * h_input;
    // 말록 함수는 void 포인터를 반환하므로 원하는 타입의 포인터로 캐스팅해야한다.
    h_input = (int*)malloc(byte_size);

    time_t t;
    srand((unsigned)time(&t));
    for(int i=0; i<size; i++) {
        h_input[i] = rand() & 0xff;
    }

    int * d_input;
    // 첫번째 인자로 더블 포인터 혹은 포인터에 포인터가 들어간다.
    cudaMalloc((void**)&d_input, byte_size);

    cudaMemcpy(d_input, h_input, byte_size, cudaMemcpyHostToDevice);

    // 총 160개의 스레드
    dim3 block(32);
    dim3 grid(5);

    mem_trs_test<<<grid, block>>>(d_input, size);
    cudaDeviceSynchronize();

    cudaFree(d_input);
    free(h_input);

    cudaDeviceReset();
    return 0;
}

/tmp/tmp8g7a0hx6/f8ed1d04-61fb-47d5-b546-47a1be17150a.cu(41): error: identifier "mem_trs_test" is undefined

1 error detected in the compilation of "/tmp/tmp8g7a0hx6/f8ed1d04-61fb-47d5-b546-47a1be17150a.cu".

